**Dataset Information: -**


1.   https://www.spectrumnews.org/news/researchers-publish-new-dataset-on-minimally-verbal-autistic-people/
2.   https://zenodo.org/record/5786860#.ZG0w_HZBw2w



**Installing files from Zenodo: -**


1.   pip install zenodo-get
2.   zenodo_get 10.5281/zenodo.5786860

(10.5281/zenodo.5786860 is the DOI of the database publication)




#Imports

In [3]:
#%tensorflow_version 2.x  # this line is not required unless you are in a notebook
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [4]:
import pandas as pd
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio

In [5]:
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio
import torch.nn.functional as F
from torch.nn import init
import torch
import torchvision
import torch.nn as nn

#Read and Prepare Dataset


In [6]:
path = "/kaggle/input/autism-paper-data/Retained/dataset_file_directory.csv"
df_original = pd.read_csv(path)
df_original.head()

#There are 6 classes of sounds in the dataset.
#The class label is categorical, and hence will be converted to a numeric Class ID later.
#E.g.: 0 = air conditioner, 1 = car horn, etc.

,Filename,Participant,Label
0,210408_2025_00-01-38.27--00-01-40.84.wav,P01,social
1,210324_2036_00-06-03.61--00-06-05.78.wav,P01,social
2,210324_2036_00-09-04.66--00-09-06.16.wav,P01,social
3,210324_2036_00-11-18.6--00-11-20.3.wav,P01,social
4,200506_2110_00-01-25.92--00-01-26.58c.wav,P01,social


Training Data: -
*   The features (X): Audio Paths
*   The target labels (y): Class Names

In [7]:
df_original.shape[0]

6450

In [8]:
df_original.Label.unique() #Print unique types of sound

array(['social', 'selftalk', 'request', 'delighted', 'dysregulated',
       'frustrated'], dtype=object)

In [9]:
df_original['Label'].nunique() #Our dataset has 22 target variables/classes.

6

In [10]:
df_original['Label'].value_counts() #Number of datapoints available for each target variable

Label
selftalk        1885
frustrated      1536
delighted       1272
dysregulated     704
social           634
request          419
Name: count, dtype: int64

In [11]:
#Idea: Consider only the top 6 majority classes and drop the rest to account for class imbalance.
#Problem: Path (file) names are random, and they are not foldered/organzied. Hence we need to write a code that will read the csv file after deletion, and pick only the un-deleted ones from the audio folder.
"""
#drop minority classes
index_names = df_original[ df_original['Label'] == "greeting" ].index
df_original.drop(index_names, inplace = True)

index_names = df_original[ df_original['Label'] == "hunger" ].index
df_original.drop(index_names, inplace = True)

index_names = df_original[ df_original['Label'] == "tablet" ].index
df_original.drop(index_names, inplace = True)

index_names = df_original[ df_original['Label'] == "laugh" ].index
df_original.drop(index_names, inplace = True)

index_names = df_original[ df_original['Label'] == "glee" ].index
df_original.drop(index_names, inplace = True)

index_names = df_original[ df_original['Label'] == "no" ].index
df_original.drop(index_names, inplace = True)

index_names = df_original[ df_original['Label'] == "dysregulation-bathroom" ].index
df_original.drop(index_names, inplace = True)

index_names = df_original[ df_original['Label'] == "bathroom" ].index
df_original.drop(index_names, inplace = True)

index_names = df_original[ df_original['Label'] == "protest" ].index
df_original.drop(index_names, inplace = True)

index_names = df_original[ df_original['Label'] == "more" ].index
df_original.drop(index_names, inplace = True)

index_names = df_original[ df_original['Label'] == "help" ].index
df_original.drop(index_names, inplace = True)

index_names = df_original[ df_original['Label'] == "happy" ].index
df_original.drop(index_names, inplace = True)

index_names = df_original[ df_original['Label'] == "dysregulation-sick" ].index
df_original.drop(index_names, inplace = True)

index_names = df_original[ df_original['Label'] == "yes" ].index
df_original.drop(index_names, inplace = True)

index_names = df_original[ df_original['Label'] == "affectionate" ].index
df_original.drop(index_names, inplace = True)

index_names = df_original[ df_original['Label'] == "laughter" ].index
df_original.drop(index_names, inplace = True)
"""

'\n#drop minority classes\nindex_names = df_original[ df_original[\'Label\'] == "greeting" ].index\ndf_original.drop(index_names, inplace = True)\n\nindex_names = df_original[ df_original[\'Label\'] == "hunger" ].index\ndf_original.drop(index_names, inplace = True)\n\nindex_names = df_original[ df_original[\'Label\'] == "tablet" ].index\ndf_original.drop(index_names, inplace = True)\n\nindex_names = df_original[ df_original[\'Label\'] == "laugh" ].index\ndf_original.drop(index_names, inplace = True)\n\nindex_names = df_original[ df_original[\'Label\'] == "glee" ].index\ndf_original.drop(index_names, inplace = True)\n\nindex_names = df_original[ df_original[\'Label\'] == "no" ].index\ndf_original.drop(index_names, inplace = True)\n\nindex_names = df_original[ df_original[\'Label\'] == "dysregulation-bathroom" ].index\ndf_original.drop(index_names, inplace = True)\n\nindex_names = df_original[ df_original[\'Label\'] == "bathroom" ].index\ndf_original.drop(index_names, inplace = True)\n\n

In [12]:
#Convert Text to number notation for target variable (Categorical to numerical data)

"""
Combine classes to account for class imbalance and to reduce the number of classes: -
self talk - 0
frusturated, protest, no - 1
delighted, laughter, happy, glee, laugh  - 2
dysregulated, dysregulation-sick, dysregulation-bathroom - 3
social, affectionate, greeting - 4
request, help, bathroom, tablet, bathroom - 5
yes, more, hunger - 6
"""

df_original["classID"] = df_original['Label']
df_original['classID'].replace(['selftalk', 'frustrated', 'delighted', 'dysregulated', 'social', 'request'],
                        [0, 1, 2, 3, 4, 5], inplace=True)


#Create new dataframe called 'df' with only two columns - Filename, classID.
df = df_original
df = df.drop(['Participant', 'Label'], axis=1)
df.head()

/tmp/ipykernel_35/4223752362.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_original['classID'].replace(['selftalk', 'frustrated', 'delighted', 'dysregulated', 'social', 'request'],
/tmp/ipykernel_35/4223752362.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_original['classID'].repl

,Filename,classID
0,210408_2025_00-01-38.27--00-01-40.84.wav,4
1,210324_2036_00-06-03.61--00-06-05.78.wav,4
2,210324_2036_00-09-04.66--00-09-06.16.wav,4
3,210324_2036_00-11-18.6--00-11-20.3.wav,4
4,200506_2110_00-01-25.92--00-01-26.58c.wav,4


In [13]:
df['classID'].value_counts() #Number of datapoints available for each target variable

classID
0    1885
1    1536
2    1272
3     704
4     634
5     419
Name: count, dtype: int64

In [14]:
# Construct file path by concatenating fold and file name
df['relative_path'] = '/' + df['Filename'].astype(str)

# Take relevant columns
df = df[['relative_path', 'classID']]
df.head()

,relative_path,classID
0,/210408_2025_00-01-38.27--00-01-40.84.wav,4
1,/210324_2036_00-06-03.61--00-06-05.78.wav,4
2,/210324_2036_00-09-04.66--00-09-06.16.wav,4
3,/210324_2036_00-11-18.6--00-11-20.3.wav,4
4,/200506_2110_00-01-25.92--00-01-26.58c.wav,4


#Audio Pre-processing

Audio data is memory intensive. So, we don’t want to read the entire dataset into memory all at once. Instead, we keep only the audio file names in our training data and perform pre-processing dynamically during runtime.

Then, at runtime, as we train the model one batch at a time, we will load the audio data for that batch and process it by applying a series of transforms to the audio. That way we keep audio data for only one batch in memory at a time.

In [15]:
#Random shuffle of rows.
df = df. sample(frac=1)
df.head()

,relative_path,classID
2725,/200605_1745_01-43-34.06--01-43-35.67.wav,3
498,/200329_1113_00-14-06.278--00-14-07.056.wav,0
3131,/200604_1725_00-12-41.29--00-12-42.51c.wav,0
2370,/200411_1548_00-55-01.84--00-55-04.2.wav,3
203,/200306_2024_00-09-52.54--00-09-52.98.wav,0


In [16]:
#Function to read and load the audio files in '.wav' format using Pytorch.
class AudioUtil():
  @staticmethod
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file) # Load an audio file. Return the signal as a tensor and the sample rate
    return (sig, sr)


#Refer tutotial file for explaination about the following blocks of code: -
  def rechannel(aud, new_channel):
    sig, sr = aud
    if (sig.shape[0] == new_channel):
      return aud
    if (new_channel == 1):
      resig = sig[:1, :]
    else:
      resig = torch.cat([sig, sig])
    return ((resig, sr))

  def resample(aud, newsr):
    sig, sr = aud
    if (sr == newsr):
      return aud
    num_channels = sig.shape[0]
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])
    return ((resig, newsr))

  def pad_trunc(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms
    if (sig_len > max_len):
      sig = sig[:,:max_len]
    elif (sig_len < max_len):
      pad_begin_len = random.randint(0, max_len - sig_len)
      pad_end_len = max_len - sig_len - pad_begin_len
      pad_begin = torch.zeros((num_rows, pad_begin_len))
      pad_end = torch.zeros((num_rows, pad_end_len))
      sig = torch.cat((pad_begin, sig, pad_end), 1)
    return (sig, sr)

  def time_shift(aud, shift_limit):
    sig,sr = aud
    _, sig_len = sig.shape
    shift_amt = int(random.random() * shift_limit * sig_len)
    return (sig.roll(shift_amt), sr)

  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

  def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec
    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
      aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)
    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
      aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)
    return aug_spec

#Define Custom Data Loader


To feed the data to a model with Pytorch, we need two objects:

*   A custom Dataset object that uses all the audio transforms to pre-process an audio file and prepare one data item at a time.
*   A built-in DataLoader object that uses the Dataset object to fetch individual data items and packages them into a batch of data.

In [17]:
class SoundDS(Dataset):
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 44100
    self.channel = 2
    self.shift_pct = 0.4

  #Number of items in dataset
  def __len__(self):
    return len(self.df)


  #Get i'th item in dataset
  def __getitem__(self, idx):
    #Absolute file path of the audio file = audio directory + relative path
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    #Get the Class ID
    class_id = self.df.loc[idx, 'classID']

    aud = AudioUtil.open(audio_file)
    """
    Some sounds have a higher sample rate, or fewer channels compared to the
    majority. So make all sounds have the same number of channels and same
    sample rate. Unless the sample rate is the same, the pad_trunc will still
    result in arrays of different lengths, even though the sound duration is
    the same.
    """
    reaud = AudioUtil.resample(aud, self.sr)
    rechan = AudioUtil.rechannel(reaud, self.channel)


    dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
    shift_aud = AudioUtil.time_shift(dur_aud, self.shift_pct)
    sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

    return aug_sgram, class_id

#Prepare Batches of Data with the Data Loader

Load the Features and Labels from our Pandas dataframe and split it in an 80:20 ratio for training and validation. Then, use them to create the training and validation Data Loaders.

In [18]:
data_path = "/kaggle/input/autism-paper-data/Retained/audio"
myds = SoundDS(df, data_path)
#myds = SoundDS(df, path)

#Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

#Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)

In [19]:
print(num_items, num_train, num_val)

6450 5160 1290


In [20]:
type(train_dl)

torch.utils.data.dataloader.DataLoader

During training, the Data Loader will randomly fetch one batch of input features containing the list of audio file names and run the pre-processing audio transforms on each audio file. It will also fetch the corresponding target labels containing the class IDs. Thus it will output one batch of training data at a time, which can directly be fed as input to our DL model.



*   The audio from the file gets loaded into a Numpy array of shape (num_channels, num_samples). Most of the audio is sampled at 44.1kHz and is about 4 seconds in duration, resulting in 44,100 * 4 = 176,400 samples. If the audio has 1 channel, the shape of the array will be (1, 176400). Similarly, audio of 4 seconds duration with 2 channels and sampled at 48kHz will have 192,000 samples and a shape of (2, 192000).
*   Since the channels and sampling rates of each audio are different, the next two transforms resample the audio to a standard 44.1kHz and to a standard 2 channels.
*   Since some audio clips might be more or less than 4 seconds, we also standardize the audio duration to a fixed length of 4 seconds. Now arrays for all items have the same shape of (2, 176,400).
*   The Time Shift data augmentation now randomly shifts each audio sample forward or backward. The shapes are unchanged.
*   The augmented audio is now converted into a Mel Spectrogram, resulting in a shape of (num_channels, Mel freq_bands, time_steps) = (2, 64, 344).
*   The SpecAugment data augmentation now randomly applies Time and Frequency Masks to the Mel Spectrograms. The shapes are unchanged.

Thus, each batch will have two tensors, one for the X feature data containing the Mel Spectrograms and the other for the y target labels containing numeric Class IDs. The batches are picked randomly from the training data for each training epoch.
Each batch has a shape of (batch_sz, num_channels, Mel freq_bands, time_steps).






# Alex Net

In [25]:
"""
import torchvision.models as models

# Load pretrained AlexNet
model = models.alexnet(pretrained=True)

# Modify the first layer to accept 2-channel input (for spectrograms)
model.features[0] = nn.Conv2d(2, model.features[0].out_channels,
                              kernel_size=model.features[0].kernel_size,
                              stride=model.features[0].stride,
                              padding=model.features[0].padding)

# Modify the last layer to match the number of output classes
num_ftrs = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(num_ftrs, 6)  # Adjust the number of classes accordingly

# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = model.to(device)

# Check that it is on Cuda
next(myModel.parameters()).device
"""

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 174MB/s]  


device(type='cuda', index=0)

In [27]:
# Define training function
def training(model, train_dl, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.0001,
                                                    steps_per_epoch=int(len(train_dl)),
                                                    epochs=num_epochs,
                                                    anneal_strategy='linear')

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_prediction = 0
        
        total_prediction = 0

        for i, data in enumerate(train_dl):
            inputs, labels = data[0].to(device), data[1].to(device)

            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            running_loss += loss.item()

            _, prediction = torch.max(outputs, 1)
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

        avg_loss = running_loss / len(train_dl)
        acc = correct_prediction / total_prediction
        print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

    print('Finished Training')


In [28]:
# Define inference function
def inference(model, val_dl):
    correct_prediction = 0
    total_prediction = 0

    with torch.no_grad():
        for data in val_dl:
            inputs, labels = data[0].to(device), data[1].to(device)

            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            outputs = model(inputs)

            _, prediction = torch.max(outputs, 1)
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

    acc = correct_prediction / total_prediction
    print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')

In [28]:
# Train the model
num_epochs = 50
training(myModel, train_dl, num_epochs)

# Run inference on the trained model
inference(myModel, val_dl)

Epoch: 0, Loss: 1.66, Accuracy: 0.30
Epoch: 1, Loss: 1.59, Accuracy: 0.34
Epoch: 2, Loss: 1.55, Accuracy: 0.37
Epoch: 3, Loss: 1.51, Accuracy: 0.38
Epoch: 4, Loss: 1.47, Accuracy: 0.40
Epoch: 5, Loss: 1.44, Accuracy: 0.42
Epoch: 6, Loss: 1.41, Accuracy: 0.44
Epoch: 7, Loss: 1.38, Accuracy: 0.45
Epoch: 8, Loss: 1.38, Accuracy: 0.45
Epoch: 9, Loss: 1.33, Accuracy: 0.49
Epoch: 10, Loss: 1.33, Accuracy: 0.48
Epoch: 11, Loss: 1.32, Accuracy: 0.49
Epoch: 12, Loss: 1.29, Accuracy: 0.50
Epoch: 13, Loss: 1.29, Accuracy: 0.50
Epoch: 14, Loss: 1.26, Accuracy: 0.51
Epoch: 15, Loss: 1.24, Accuracy: 0.52
Epoch: 16, Loss: 1.21, Accuracy: 0.54
Epoch: 17, Loss: 1.19, Accuracy: 0.55
Epoch: 18, Loss: 1.18, Accuracy: 0.55
Epoch: 19, Loss: 1.16, Accuracy: 0.56
Epoch: 20, Loss: 1.14, Accuracy: 0.57
Epoch: 21, Loss: 1.11, Accuracy: 0.58
Epoch: 22, Loss: 1.09, Accuracy: 0.59
Epoch: 23, Loss: 1.08, Accuracy: 0.59
Epoch: 24, Loss: 1.05, Accuracy: 0.60
Epoch: 25, Loss: 1.05, Accuracy: 0.60
Epoch: 26, Loss: 1.02,

In [29]:
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np

def inference(model, val_dl):
    all_predictions = []
    all_labels = []
    
    # Disable gradient updates
    with torch.no_grad():
        for data in val_dl:
            inputs, labels = data[0].to(device), data[1].to(device)

            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Get predictions
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
    f1 = f1_score(all_labels, all_predictions, average='weighted')
        
    print(f'Accuracy: {accuracy:.2f}, F1 Score: {f1:.2f}')

# Run inference on trained model with the validation set
inference(myModel, val_dl)

Accuracy: 0.65, F1 Score: 0.64


# MobileVNet

In [21]:
import torch
import torchvision.models as models
import torch.nn as nn

# Load a pretrained MobileNetV3 Large model
model = models.mobilenet_v3_small(pretrained=True)

# Modify the first layer to accept 2-channel input
first_conv_layer = model.features[0][0]
model.features[0][0] = nn.Conv2d(2, first_conv_layer.out_channels,
                                 kernel_size=first_conv_layer.kernel_size,
                                 stride=first_conv_layer.stride,
                                 padding=first_conv_layer.padding)


num_classes = 6
# Replace the classifier head with a new one adjusted for your number of classes
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)

# Check if the model should be trained on a GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = model.to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 43.6MB/s]


In [22]:
# Define training function
def training(model, train_dl, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                    steps_per_epoch=int(len(train_dl)),
                                                    epochs=num_epochs,
                                                    anneal_strategy='linear')

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_prediction = 0
        total_prediction = 0

        for i, data in enumerate(train_dl):
            inputs, labels = data[0].to(device), data[1].to(device)

            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            running_loss += loss.item()

            _, prediction = torch.max(outputs, 1)
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

        avg_loss = running_loss / len(train_dl)
        acc = correct_prediction / total_prediction
        print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

    print('Finished Training')

In [23]:
# Define inference function
def inference(model, val_dl):
    correct_prediction = 0
    total_prediction = 0

    with torch.no_grad():
        for data in val_dl:
            inputs, labels = data[0].to(device), data[1].to(device)

            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            outputs = model(inputs)

            _, prediction = torch.max(outputs, 1)
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

    acc = correct_prediction / total_prediction
    print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')

In [24]:
# Train the model
num_epochs = 50
training(myModel, train_dl, num_epochs)

# Run inference on the trained model
inference(myModel, val_dl)

Epoch: 0, Loss: 1.64, Accuracy: 0.32
Epoch: 1, Loss: 1.50, Accuracy: 0.40
Epoch: 2, Loss: 1.41, Accuracy: 0.45
Epoch: 3, Loss: 1.33, Accuracy: 0.49
Epoch: 4, Loss: 1.28, Accuracy: 0.51
Epoch: 5, Loss: 1.23, Accuracy: 0.53
Epoch: 6, Loss: 1.20, Accuracy: 0.54
Epoch: 7, Loss: 1.16, Accuracy: 0.56
Epoch: 8, Loss: 1.14, Accuracy: 0.57
Epoch: 9, Loss: 1.11, Accuracy: 0.59
Epoch: 10, Loss: 1.10, Accuracy: 0.59
Epoch: 11, Loss: 1.09, Accuracy: 0.59
Epoch: 12, Loss: 1.07, Accuracy: 0.60
Epoch: 13, Loss: 1.05, Accuracy: 0.61
Epoch: 14, Loss: 1.05, Accuracy: 0.62
Epoch: 15, Loss: 1.04, Accuracy: 0.62
Epoch: 16, Loss: 1.03, Accuracy: 0.62
Epoch: 17, Loss: 0.99, Accuracy: 0.64
Epoch: 18, Loss: 0.95, Accuracy: 0.65
Epoch: 19, Loss: 0.95, Accuracy: 0.65
Epoch: 20, Loss: 0.92, Accuracy: 0.66
Epoch: 21, Loss: 0.90, Accuracy: 0.67
Epoch: 22, Loss: 0.86, Accuracy: 0.68
Epoch: 23, Loss: 0.83, Accuracy: 0.69
Epoch: 24, Loss: 0.83, Accuracy: 0.69
Epoch: 25, Loss: 0.78, Accuracy: 0.71
Epoch: 26, Loss: 0.79,

In [25]:
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np

def inference(model, val_dl):
    all_predictions = []
    all_labels = []
    
    # Disable gradient updates
    with torch.no_grad():
        for data in val_dl:
            inputs, labels = data[0].to(device), data[1].to(device)

            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Get predictions
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
    f1 = f1_score(all_labels, all_predictions, average='weighted')
        
    print(f'Accuracy: {accuracy:.2f}, F1 Score: {f1:.2f}')

# Run inference on trained model with the validation set
inference(myModel, val_dl)

Accuracy: 0.69, F1 Score: 0.69
